# Homework 8: I/O and data structures practice

## Goal

In this assignment we will practice working with files, more practice with dictionaries, and performing some basic geometric computations. Recall earlier in the course I walked through an example of processing retail data where we
had data about a set of products in a store, and the set of baskets that people bought.  We will work
with that and extend it a little for this assignment.  Our working data will be the following:

- _Product inventory_: This will be a table of products where each row corresponds to one product, and each column represents a property of the objects.  Specifically, our product inventory table will have five columns: a unique product ID, a text description of the product, a unit price in dollars, an X coordinate, and a Y coordinate.

- _Sales data_: This will be a table of records from the point of sale system.  Each row will correspond to an item in a basket.  The columns will be the basket ID, the product ID, product quantity, and the product pick up order.  The product pick up order for a basket containing $n$ items will range from 1 to $n$ and corresponds to the order that the customer picked up the products as they went through the store.

We will also be provided additional parameters about the store.  Specifically, we will be given the X,Y
coordinates of the entry door, the checkout stand, and the exit door.  We are going to assume that this
store has only one checkout line and all customers are good at following signs and always use the appropriate door for entry and exit.  We are also going to assume that customers have the magical ability to teleport through shelves so that they can take the shortest path from one product to the next, allowing us to avoid worrying about complex calculations of the distance between any two products.

The coordinates of the entrance, exit, and checkout are:

In [1]:
store_entrance = (10,0)
store_exit = (90,0)
store_checkout = (50,10)

## Sample I/O code

To get you started, here is code that reads in the inventory data from the CSV file and produces a dictionary of dictionaries.  The outer dictionary maps product IDs as strings to the inner dictionary, and the inner dictionary for each product maps an attribute (e.g., 'unit_price') to its value.  Numerical values are stored as floating point numbers, _not strings_.

In [2]:
import csv

In [3]:
def read_inventory(filename):
    # initialize empty dict
    inventory={}
    
    # open the given file and name the file handle f
    with open(filename) as f:
        # create a CSV reader object from the file
        reader = csv.reader(f)
        
        # advance the reader to skip the first header line
        next(reader)
        
        # for each row in the CSV file, create the appropriate
        # entry in the inventory.  this includes converting the
        # strings for price, x, and y into floats so we can do
        # arithmetic with them later.
        for row in reader:
            inventory[row[0]] = { 'desc':row[1],
                                  'unit_price':float(row[2]),
                                  'x':float(row[3]),
                                  'y':float(row[4]) }
            
        return inventory

In [4]:
inventory = read_inventory('inventory.csv')

Test it: what is the price of product ID 4?

In [5]:
inventory['4']['unit_price']

0.39

## Part 1: read the basket data

Complete the following function to return a dictionary mapping basket ID to some data structure of your choice to represent the basket contents.

In [15]:
def read_baskets(filename):
    baskets = {}
    f = open(filename)
    lines = f.readlines()
    for line in lines[1:]:
        parts = line[:-1].split(',')
        
        if parts[0] in baskets:
            baskets[parts[0]].append( (parts[1], int(parts[2]), int(parts[3]) ) )
        else:
            baskets[parts[0]] = [(parts[1], int(parts[2]), int(parts[3]))]
            
    f.close()
    for item in baskets:
        baskets[item] = sorted(baskets[item], key = lambda x:x[2])
    return baskets

In [16]:
baskets = read_baskets('baskets.csv')
baskets

{'1': [('1', 2, 1), ('3', 1, 2)],
 '10': [('1', 1, 1), ('3', 2, 2), ('7', 1, 3), ('5', 2, 4)],
 '2': [('1', 3, 1), ('4', 1, 2), ('3', 1, 3), ('11', 3, 4)],
 '3': [('6', 2, 1), ('5', 2, 2), ('13', 1, 3)],
 '4': [('7', 1, 1),
  ('10', 2, 2),
  ('3', 1, 3),
  ('8', 1, 4),
  ('12', 2, 5),
  ('9', 1, 6)],
 '5': [('2', 8, 1), ('4', 8, 2), ('3', 8, 3), ('1', 8, 4)],
 '6': [('3', 2, 1), ('4', 2, 2), ('1', 2, 3), ('2', 1, 4)],
 '7': [('13', 1, 1), ('9', 1, 2)],
 '8': [('7', 1, 1)],
 '9': [('8', 3, 1), ('6', 1, 2), ('4', 4, 3), ('12', 1, 4), ('7', 1, 5)]}

## Part 2: Calculate the path length of each customer

Given the basket data and inventory data, write a function that calculates the distance traveled by a customer through the store.  Their trip should go entry -> first product -> second product -> ... -> checkout -> exit.  You should assume that the customer takes a straight path from each point to the next.

In [17]:
from math import sqrt
def Euclid_Distance(pt1,pt2):
    x_diff=pt1[0]-pt2[0]
    y_diff=pt1[1]-pt2[1]
    return sqrt(x_diff**2 + y_diff**2)

In [27]:
def customer_trip(inventory, baskets, basket_id):
    dist=0
    pts=[]
    pts.append(store_entrance)

    for item in baskets [basket_id]:
        product = item[0]
        pts.append((inventory[product]['x'], inventory[product]['y']))
    pts.append(store_checkout)
    pts.append(store_exit)

    
    for d in range(1,len(pts)):
        dist += Euclid_Distance(pts[d],pts[d-1])
    return dist

In [29]:
basket_id = '4'
customer_trip(inventory, baskets, basket_id)

373.4229596034474

## Part 3: Calculate the total price for each basket

Given the basket and inventory data, write a function that calculates the total cost of a basket.

In [30]:
def basket_total(inventory, baskets, basket_id):
    total_price=0
    for item in baskets [basket_id]:
        product=item[0]
        quantity=item[1]
        total_price += inventory[product]['unit_price']*quantity
    return total_price

In [31]:
basket_id = '4'
basket_total(inventory, baskets, basket_id)

29.43

## Part 4: Calculate the price per unit of distance traveled for all baskets

For each basket we have a distance traveled and a total price.  Write a function that returns a dictionary mapping the basket ID to the price per unit distance travelled.

In [48]:
def price_distance_calculate(inventory, baskets):
    price_distance_dict = {}
    for key in baskets:
        price = basket_total(inventory, baskets, key)
        distance = customer_trip(inventory, baskets, key)
        price_distance = price/distance
        price_distance_dict[int(key)]= price_distance
    return price_distance_dict

In [49]:
price_distance_calculate(inventory, baskets)

{1: 0.0091229257242453,
 2: 0.06942077638079959,
 3: 0.09012356902592947,
 4: 0.07881143685233731,
 5: 0.07312101528805648,
 6: 0.017302536628741667,
 7: 0.055204158287642274,
 8: 0.00905699668841218,
 9: 0.06571221581998327,
 10: 0.032153884764837576}

## Part 5: EXTRA CREDIT.  

### Calculate the difference between the length of the path each customer took versus the shortest path they could have taken.

Each customer may have travelled the store inefficiently.  We would like to know the excess distance each customer covered versus what they could have done had they planned their trip more carefully.  Write a function that calculates the shortest path that a customer could have taken.

In [ ]:
def customer_shortest_trip(inventory, baskets, basket_id):
    # fill me in
    pass